In [1]:
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchvision
import torchvision.utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import os

from dataset import next_batch

In [2]:
class SiameseNetwork(nn.Module):
  
  def __init__(self, 
               dropout=0.2, 
               output_size=5):
    
    super(SiameseNetwork, self).__init__()
    
    self.dropout = dropout
    self.output_size = output_size
    
    self.cnn = nn.Sequential(
        nn.ReflectionPad2d(1),
        nn.Conv2d(1, 4, kernel_size=3),
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(4),
        nn.Dropout2d(p=dropout),
        
        nn.ReflectionPad2d(1),
        nn.Conv2d(4, 8, kernel_size=3),
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(8),
        nn.Dropout2d(p=dropout),
        
        nn.ReflectionPad2d(1),
        nn.Conv2d(8, 8, kernel_size=3),
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(8),
        nn.Dropout2d(p=dropout),
    )
    
    self.dense = nn.Sequential(
        nn.Linear(8 * 100 * 100, 500),
        nn.ReLU(inplace=True),
        nn.Linear(500, 500),
        nn.ReLU(inplace=True),
        nn.Linear(500, output_size) 
    )
    
  def forward_once(self, x):
    output = self.cnn(x)
    output = output.view(output.size()[0], -1)
    return self.dense(output)
  
  
  def forward(self, input_1, input_2):
    output_1 = self.forward_once(input_1)
    output_2 = self.forward_once(input_2)
    return output_1, output_2

In [3]:
class ContrastiveLoss(nn.Module):
  """
  Contrastive loss function.
  Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
  """

  def __init__(self, margin=2.0):
    super(ContrastiveLoss, self).__init__()
    self.margin = margin

  def forward(self, output1, output2, label):
    euclidean_distance = F.pairwise_distance(output1, output2)
    loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                  (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


    return loss_contrastive

In [ ]:
def to_var(x):
  if torch.cuda.is_available():
    x = x.cuda()
  return Variable(x)


def denorm(x):
  out = (x + 1) / 2
  return out.clamp(0, 1)


def imshow(img, text=None, size=8):
  
  np_img = img.numpy()
  width, height = np_img.shape
  
  fig = plt.figure(figsize=(size * height / width, size))
  
  plt.axis("off")
  if text:
    plt.text(75, 8, text, style='italic', fontweight='bold',
             bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
  plt.imshow(np_img)
  plt.show() 

In [ ]:
dataset_folder = './sign_dataset/'

embedding_size = 2
learning_rate = 0.001

training_iterations = 5000
save_step = 100
batch_size = 4

image_height = 64
image_width = 64
image_channels = 3
target_shape = (image_height, image_width, image_channels)

amount_test_batches = 20
amount_test_images = batch_size * amount_test_batches

batch_generator = next_batch(batch_size,
                             data_directory=dataset_folder,
                             target_shape=target_shape,
                             probability=0.1)

# Load model and loss.
if torch.cuda.is_available():
    network   = SiameseNetwork(output_size=embedding_size).cuda()
    criterion = ContrastiveLoss().cuda()
    optimiser = optim.Adam(network.parameters(), lr=learning_rate)

    for batch_left, batch_right, batch_similar in batch_generator:

        # Initialise the gradients buffers.
        optimiser.zero_grad()

        # Convert to variables on the GPU.
        batch_left, batch_right, batch_similar = \
            torch.from_numpy(batch_left), torch.from_numpy(batch_right), torch.from_numpy(batch_similar)
        batch_left, batch_right, batch_similar = \
            to_var(batch_left), to_var(batch_right), to_var(batch_similar)

        # Get encoding vectors.
        encoding_left, encoding_right = network(batch_left, batch_right)

        # Loss calculated and backpropagated.
        loss = criterion(encoding_left, encoding_right, is_different)
        loss.backward()
        optimiser.step()

        print("loss: {}".format(loss.data[0]))

else:
    print("Please run on hardware with GPU support")

/usr/local/lib/python3.5/dist-packages/torch/cuda/__init__.py:97: UserWarning: 
    Found GPU0 Quadro M1000M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


Initialised with 296 image(s) found.
Output directory set to ./sign_dataset/kfh/output.Initialised with 370 image(s) found.
Output directory set to ./sign_dataset/acorn/output.Initialised with 249 image(s) found.
Output directory set to ./sign_dataset/kallars/output.Initialised with 846 image(s) found.
Output directory set to ./sign_dataset/hamptons/output.Initialised with 435 image(s) found.
Output directory set to ./sign_dataset/bryan_keegan/output.Initialised with 1014 image(s) found.
Output directory set to ./sign_dataset/ocean/output.Initialised with 342 image(s) found.
Output directory set to ./sign_dataset/barnard_marcus/output.